In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-08-28"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
50,2024-08-28,Eua Wnba,20:00,Indiana Fever F,Connecticut Sun F,2.45,1.56,164.5,1.86,1.92,2.5,2.03,1.33,na7M7gBB,0.408163,0.641026,0.537634,0.520833,0.049189,0.8,0.6,NaN,NaN,167.478,28.63063,0.170952,146.134,42.887675,0.293482,182.946,139.618,207.10,213.12,0.0,1.0,0.0,0.0,0.313878,0.022448,0.294628,3.35,0.670,2.164179,0.0,0.0,0.0,0.97,0.194,2.886598,0.0,0.0,0.0
51,2024-08-28,Eua Wnba,21:00,Chicago Sky F,Washington Mystics F,1.95,1.85,158.5,1.80,1.96,1.5,1.80,1.69,02BQ6DQH,0.512821,0.540541,0.555556,0.510204,0.053361,0.0,0.0,NaN,NaN,235.648,124.70509,0.529201,230.268,135.848249,0.589957,0.000,0.000,448.20,145.96,1.0,0.0,0.0,0.0,0.037216,0.060179,0.044574,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
52,2024-08-28,Eua Wnba,23:00,Phoenix Mercury F,Minnesota Lynx F,2.45,1.56,162.5,1.81,1.97,2.5,2.03,1.42,MHaZ4ieU,0.408163,0.641026,0.552486,0.507614,0.049189,0.6,0.0,NaN,NaN,138.328,17.46179,0.126235,182.370,54.310607,0.297805,159.656,0.000,142.56,166.32,0.0,0.0,0.0,0.0,0.313878,0.059861,0.250049,0.59,0.118,12.288136,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
53,2024-08-28,Mundo Amistoso Interclubes,15:00,Manresa (Esp),Basquet Girona (Esp),2.04,1.66,163.5,1.82,1.84,1.5,1.92,1.58,nNTI1Wft,0.490196,0.602410,0.549451,0.543478,0.092606,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.145244,0.007728,0.137381,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
54,2024-08-28,Mundo Amistoso Interclubes,12:30,Juventus (Ltu),Rytas (Ltu),3.44,1.28,167.5,1.83,1.87,7.5,2.04,1.69,IgUC4MKF,0.290698,0.781250,0.546448,0.534759,0.071948,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.647182,0.015289,0.132701,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
55,2024-08-28,Mundo Amistoso Interclubes,14:00,Le Mans (Fra),Ada Blois (Fra),1.14,5.09,158.5,1.83,1.87,-13.5,2.01,1.70,vRuaXBfJ,0.877193,0.196464,0.546448,0.534759,0.073657,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.896652,0.015289,0.118169,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
56,2024-08-28,Mundo Amistoso Interclubes,14:45,Breidablik (Ice),Haukar (Ice),12.00,1.01,181.5,1.83,1.87,10.5,2.04,1.68,QLpJx7Wb,0.083333,0.990099,0.546448,0.534759,0.073432,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,1.194635,0.015289,0.136859,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
57,2024-08-28,Mundo Amistosos Interclubes,14:00,La Roche F (Fra),Landes F (Fra),2.86,1.35,134.5,1.83,1.83,5.5,1.98,1.70,GneZxGQs,0.349650,0.740741,0.546448,0.546448,0.090391,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.507236,0.000000,0.107603,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0


## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

Sem jogos
